# 단순 회귀 분석
5장. 통계에서 correlation 함수를 사용해 두 변수의 선형 관계를 계산해 보았다. 대부분의 경우, 선형 관계의 존재 여부를 확인하는 것만으로는 충분하지 않다.

## 14.1 모델
앞서 예시로 든 사용자의 친구 수와 사용자가 사이트에서 보낸 시간의 관계를 살펴보았는데, 친구 수가 많은 것이 사용자의 사이트 체류 시간을 더 길게 만든다는 가설에 어느 정도 확신하게 됐다고 가정하자. \
이러한 관계는 상단히 강한 선형 관계를 지니고 있고 시작점은 당연히 선형 모델이 되어야 한다. \
위의 관계를 수식으로 표현하면 

$y_i = \beta {x}_i + \alpha + {\epsilon}_i$

여기서 $y_i$는 사용자 i가 매일 사이트에서 보내는 시간(분)을 의미, $x_i$는 사용자 i의 친구 수를 나타낸다. $\epsilon$(엡실론)은 모델이 고려하지 못하는 다른 요소 때문에 밸생하는 (작은) 오류를 의미한다. 

만약, $\alpha$, $\beta$가 이미 구해졌다고 가정하면, 다음과 같은 함수를 통해 간단하게 예측을 할 수 있다.

In [1]:
def predict(alpha: float, beta: float, x_i: float) -> float:
    return beta * x_i + alpha

$\alpha$, $\beta$는 어떻게 선택할 수 있을까? \
$\alpha$, $\beta$가 무엇이든 간에 입력 변수 x_i에 대한 결과를 예측할 수 있다. 실제 출려값인 y_i가 주어졌으니, 다양한 $\alpha$, $\beta$에 대한 오류를 계산할 수 있다.

In [2]:
def error(alpha: float, beta: float, x_i: float, y_i: float) -> float:
    """
    실제 결과 y_i일때,
    beta * x_i + alpha로 계산된 예측값의 오류
    """
    return predict(alpha, beta, x_i) - y_i

**데이터 전체에서 발생하는 총 오륫값(error)를 구하기 위해서 무작정 모든 오륫값을 더해서느 안된다.** \
만약, x_1의 예측값이 너무 높고 x_2의 예측값이 너무 낮다면, 오륫값이 서로 **상쇄**되기 때문이다. 

$\Rightarrow$ **오류의 제곱 값**을 더해줘야 한다.

In [3]:
from typing import List

Vector = List[float]

def sum_of_sqerrors(alpha: float, beta: float, x: Vector, y: Vector) -> float:
    return sum(error(alpha, beta, x_i, y_i) ** 2 for x_i, y_i in zip(x, y))

**최소자승법(least squares)** 란 sum_of_sqerrors를 **최소화** 하는 $\alpha$, $\beta$ 값을 찾는 것을 의미 \
미분 사용하면 오류를 최소화하는 알파와 베타를 찾을 수 있다.

In [4]:
from typing import Tuple
import math

Vector = List[float]

def dot(v: Vector, w: Vector) -> float:
    """v_1 * w_1 + ... + v_n * w_n"""
    assert len(v) == len(w),  "vectors must be same length"
    
    return sum(v_i * w_i for v_i, w_i in zip(v,w))

def sum_of_squares(v: Vector) -> float:
    """v_1 * v_1 + ... v_n * v_n"""
    return dot(v,v)

def mean(xs: List[float]) -> float:
    return sum(xs) / len(xs)

def de_mean(xs: List[float]) -> List[float]:
    """x의 모든 데이터 포인트에서 평균을 뺌(평균을 0으로 만들기 위해)"""
    x_bar = mean(xs)
    return [x - x_bar for x in xs]

def variance(xs: List[float]) -> float:
    """편차의 제곱의 평균"""
    assert len(xs) >= 2, "variance requires at least two elements"
    
    n = len(xs)
    deviations = de_mean(xs)
    return sum_of_squares(deviations) / (n - 1)

def standard_deviation(xs: List[float]) -> float:
    """표준편차는 분산의 제곱근"""
    return math.sqrt(variance(xs))

def covariance(xs: List[float], ys: List[float]) -> float:
    assert len(xs) == len(ys),  "xs and ys must have same number of elements"
    return dot(de_mean(xs), de_mean(ys)) / (len(xs) - 1)

def correlation(xs: List[float], ys: List[float]) -> float:
    """xs와 ys의 값이 각각의 평균에서 얼마나 멀리 떨어져 있는지 계산"""
    stdev_x = standard_deviation(xs)
    stdev_y = standard_deviation(ys)
    if stdev_x > 0 and stdev_y > 0:
        return covariance(xs, ys) / stdev_x / stdev_y
    else:
        return 0  #  편차가 존재하지 않는다면 상관관계는 0

In [5]:
def least_squares_fit(x: Vector, y: Vector) -> Tuple[float, float]:
    """x와 y가 학습 데이터로 주어졌을 때
    오류의 제곱 값을 최소화해 주는 알파와 베타를 계산
    """
    beta = correlation(x, y) * standard_deviation(y) / standard_deviation(x)
    alpha = mean(y) - beta * mean(x)
    return alpha, beta

이 방법은 말이 안된다. 독립 변수 x의 평균이 주어지면, 알파는 종속 변수 y의 평균을 예측해준다. \
베타는 입력 변수가 standard_deviation(x)만큼 증가한다면 예측값 또한 correlation(x, y) * standard_deviation(y) 만큼 증가한다는 것을 의미한다. x와 y가 완벽한 양의 상관관계를 지닌다면, x가 1 표준편차 만큼 증가할 때마다 y또한 y의 1 표준편차 만큼 증가한다. 둘이 완벽한 음의 상관간계를 지닌다면, x가 증가할때마다 y는 감소한다. 상관관계가 0이라면 베타는 0이 될 것이며, 이는 x가 예측에 아무런 영향이 없다는 것을 의미한다.

In [6]:
# 간단한 데스트를 해보자
x = [i for i in range(-100, 110, 10)]
y = [3 * i - 5 for i in x]

# y = 3x - 5를 찾아내야 한다.
assert least_squares_fit(x, y) == (-5, 3)

In [8]:
# 5장. 통계, import num_friends_good, daily_minutes_good
alpha, beta = least_squares_fit(num_friends_good, daily_minutes_good)
assert 22.9 < alpha < 23.0
assert 0.9 < beta < 0.905

NameError: name 'num_friends_good' is not defined

In [10]:
from typing import List

def mean(xs: List[float]) -> float:
    return sum(xs) / len(xs)

def de_mean(xs: List[float]) -> List[float]:
    """x의 모든 데이터 포인트에서 평균을 뺌(평균을 0으로 만들기 위해)"""
    x_bar = mean(xs)
    return [x - x_bar for x in xs]

In [11]:
def total_sum_of_squares(y: Vector) -> float:
    """평균을 기준으로 y_i의 변화량을 제곱한 값의 총합"""
    return sum(v ** 2 for v in de_mean())

def r_squared(alpha: float, beta: float, x: Vector, y: Vector) -> float:
    """
    모델이 잡아낼 수 있는 y의 변화량의 비율은 
    1 - 모델이 잡아내지 못하는 y의 변화량의 비율로 계산할 수 있다.
    """
    return 1.0 - (sum_of_sqerrors(alpha, beta, x, y) / total_sum_of_squares(y))

In [12]:
rsq = r_squared(alpha, beta, num_friends_good, daily_minutes_good)
assert 0.328 < rsq < 0.330

NameError: name 'alpha' is not defined

오류의 제곱 값을 최소화하는 알파와 베타를 구했다. 모든 예측값을 항상 mean(y)로 계산해 주는 다른 모델(알파 = mean(y), 베타 = 0)을 만들 수 있다. \
이런 경우, 오류를 제곱한 값의 총합은 항상 변화량을 제곱한 값의 총합과 같은 것이며, R 제곱 값은 0이 될 것이다. 즉, 오류의 제곱 값을 최소화하는 모델을 항상 mean(y)로 예측하는 것과 별다른 차이가 없다는 것을 의미한다. \
최소자승법 모델이 성능은 적어도 평균을 예측하는 모델의 성능만큼은 좋아야한다. 즉, 오류를 제곱한 값의 총합은 아무리 커봐야 변화량을 제곱한 값의 총합과 동일할 것이다. 이 경우 R 제곱 값은 적어도 0은 될 것이다. 그리고 오류를 제곲한 값의 총합은 최소한 0이기 때문에 R 제곱 값의 최댓값은 1이다. \
R 제곱 값이 클수록 모델이 데이터에 더 적합하다는 것을 의미한다. 계산된 R 제곱 값은 0.329이기 때문에 만들어진 모델은 어느 정도 데이터 적합하지만 다른 중요한 요소가 존재한다는 것을 의미한다.

## 14.2 경사 하강법 사용하기
만약, theta = [alpha, beta]로 설정하면 경사 하강법을 통해 모델을 만들 수 있다.

In [14]:
import random
import tqdm

def scalar_multiply(c: float, v: Vector) -> Vector:
    """모든 성분을 c로 곱하기"""
    return [c * v_i for v_i in v]

def gradient_step(v: Vector, gradient: Vector, step_size: float) -> Vector:
    """v에서 step_size만큼 이동하기"""
    assert len(v) == len(gradient)
    step = scalar_multiply(step_size, gradient)
    return add(v, step)

In [16]:
num_epochs = 10000
random.seed(0)

guess = [random.random(), random.random()]  # 임의의 위치에서 출발
learning_rate = 0.00001

with tqdm.trange(num_epochs) as t:
    for _ in t:
        alpha, beta = guess
        # 알파에 대한 손실 함수의 편미분
        grad_a = sum(2 * error(alpha, beta, x_i, y_i)
                    for x_i, y_i in zip(num_friends_good, daily_minutes_good))
        # 베타에 대한 손실 함수의 편미분
        grad_b = sum(2 * error(alpha, beta, x_i, y_i) * x_i
                    for x_i, y_i in zip(num_friends_good, daily_minutes_good))
        # tqdm 설명에 넣기 위해 손실 계산
        loss = sum_of_sqerrors(alpha, beta, num_friends_good, daily_minutes_good)
        t.set_description(f"loss: {loss:.3f}")
        
        # 최종적으로, 추측을 갱신한다.
        guess = gradient_step(guess, [grad_a, grad_b], -learning_rate)

  0%|          | 0/10000 [00:00<?, ?it/s]


NameError: name 'num_friends_good' is not defined

In [17]:
# 비슷한 결과를 얻어야 한다.
alpha, beta = guess
assert 22.9 < alpha < 23.0
assert 0.9 < beta < 0.905

AssertionError: 

## 최대가능도 추정법
최소자승법을 사용하는 이유는 무엇일까? 한 가지 이유는 **최대가능도추정법(maximum likelihood estimation, MLE)** 때문이다. \
가령 임의의 $\theta$에 의존하는 분포에서 $v_1, ... v_n$라는 표본 데이터가 주어졌다고 해보자. \
$p(v_1, ... ,v_n|\theta)$ \
$\theta$를 모른다면, 위의 확률을 표본이 주어졌을 때 $\theta$가 발생할 가능도(likelihood)로 바꿔서 생각해 볼 수 있다. \
$L(\theta | v_1, ..., v_n)$ \
이 경우 가장 적절한 $\theta$는 가능도를 최대화해 주는 값이다. 즉, 관측된 데이터가 발생할 경우를 가장 높게 만들어 주는 값이라는 의미이다. \
확률 질량 함수 대신 확률 밀도 함수를 사용하는 연속형 분포에도 이를 동일하게 적용할 수 있다. \
회귀 분석으로 다시 돌아가보면 대부분의 회귀 분석에서는 오류를 평균이 0이고 표준편차가 $\sigma$인 정규분포를 따른다고 가정한다. \
이 경우 (x_i, y_i)가 관측될 가능도는 다음과 같다. \
$L(\alpha, \beta|x_i, y_i, \sigma) = \frac{1}{\sqrt{2\pi\sigma}}exp(-{(-y_i - \alpha - \beta x_i)}^2 / 2{\sigma}^2)$ \
전체 데이터에 대한 가능도는 각 데이터의 가능도를 모두 곱한 값이다. 그리고 오류의 제곱 값을 최소화하는 알파와 베타가 계산되는 지점이 가능도가 최대화 되는 지점이다. 즉, 이런 가정을 따르면 오류의 제곱 값을 최소화하는 것은 관측된 데이터가 발생할 가능도를 최대화하는 것과 동일하다.